In [ ]:
!pip install datasets
!pip install evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset=load_dataset('opus_books','en-fr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [ ]:
dataset=dataset['train'].train_test_split(0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 25417
    })
})

In [ ]:
dataset['train'][1]

{'id': '29428',
 'translation': {'en': 'He has never been seen since, but it is said that he reached America."',
  'fr': "On ne l'a jamais revu depuis, mais on dit qu'il a gagné l'Amérique."}}

In [ ]:
from transformers import AutoTokenizer
checkpoint='google-t5/t5-small'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
source_lang='en'
target_lang='fr'
prefix='translate English to French:'
def preprocess_text(examples):
  inputs=[prefix+example[source_lang] for example in examples['translation']]
  targets=[example[target_lang] for example in examples['translation']]
  tokenized_inputs=tokenizer(inputs,max_length=128,truncation=True)
  with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

  tokenized_inputs["labels"] = labels["input_ids"]
  return tokenized_inputs


In [ ]:
preprocessed_dataset=dataset.map(preprocess_text,batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [ ]:
preprocessed_dataset['train'][1]

{'id': '29428',
 'translation': {'en': 'He has never been seen since, but it is said that he reached America."',
  'fr': "On ne l'a jamais revu depuis, mais on dit qu'il a gagné l'Amérique."},
 'input_ids': [13959,
  1566,
  12,
  2379,
  10,
  3845,
  65,
  470,
  118,
  894,
  437,
  6,
  68,
  34,
  19,
  243,
  24,
  3,
  88,
  3495,
  1371,
  535,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [461,
  3,
  29,
  15,
  3,
  40,
  31,
  9,
  6462,
  5109,
  76,
  2873,
  6,
  971,
  30,
  1227,
  17,
  546,
  31,
  173,
  3,
  9,
  31397,
  3,
  40,
  31,
  27520,
  5,
  1]}

In [ ]:
from transformers import DataCollatorForSeq2Seq
datacollator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=checkpoint,return_tensors='tf')

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM
model=TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
train_set=model.prepare_tf_dataset(
    preprocessed_dataset['train'],
    batch_size=16,
    shuffle=True,
    collate_fn=datacollator
)
val_set=train_set=model.prepare_tf_dataset(
    preprocessed_dataset['test'],
    batch_size=16,
    shuffle=False,
    collate_fn=datacollator
)

In [ ]:
from tensorflow import keras
model.compile(keras.optimizers.Adam(2e-5))

In [ ]:
import evaluate
metric=evaluate.load('sacrebleu')

In [ ]:
import numpy as np
def compute_metrics(p):
  preds,labels=p
  if isinstance(preds,tuple):
    preds=preds[0]
  decoded_preds=tokenizer.decode_batch(preds,skip_special_tokens=True)
  decoded_preds=[preds.strip() for preds in decoded_preds]
  labels=np.where(labels!=-100,labels,tokenizer.pad_token_id)
  decoded_labels=tokenizer.decode_batch(labels,skip_special_tokens=True)
  decoded_labels=[[label.strip()] for label in decoded_labels]
  result=metric.compute(predictions=decoded_preds,references=decoded_labels)
  result={'bleu':result['score']}
  predicted_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in preds]
  result['gen_len']=np.mean(predicted_lens)
  return result


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
callback=KerasMetricCallback(metric_fn=compute_metrics,eval_dataset=val_set,predict_with_generate=True,use_xla_generation=True,
    generate_kwargs={"max_length": 128})

In [ ]:
model.fit(train_set,epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  14/1589 [..............................] - ETA: 10:52:30 - loss: 2.3726

In [ ]:
input_text='translate English to French: My name is Dr. Jasmeet Singh. I teach Conversational AI'
input_ids=tokenizer(input_text,return_tensors='tf').input_ids
output=model.generate(input_ids)
tokenizer.decode(output[0],skip_special_tokens=True)